In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import sqlalchemy
import os

In [ ]:
user_name = 'KHU9683'
pword = 'Mer82@dme'
host_name='EDWPROD.DW.MEDCITY.NET'

In [ ]:
td_engine = sqlalchemy.create_engine('teradatasql://' + host_name + '/?user=' + user_name + '&password=' + pword + '&logmech=LDAP')

In [ ]:
#path of file with table name and database name in csv file 
output_path_folder = "OutputFiles\\DDL\\"
td_output_path_folder = "OutputFiles\\TD_DDL\\"
output_file_extension = "csv"
schema_prefix = "edwcr"

In [ ]:
#Write file to local directory
def write_file_local(path,file_data):
    
    with open(path, 'w') as file:
        file_string = '\n'.join(file_data)
        file.write(file_string)

In [ ]:
#get the DDLs for the tables from teradata 
def gen_db_objects_list_lob():
        dt1 = datetime.now()

        if not os.path.exists(output_path_folder):
            os.makedirs(output_path_folder)      

        try:
            query = "Select Lower(DataBaseName) as database_name, Lower(TableName) as table_name, "\
            "TableKind as object_type, RequestText as view_definition From dbc.TablesV "\
            "Where TableKind IN ('T','V','O') and Lower(DataBaseName) like '{}%' "\
            "and (INSTR(UPPER(TableName),'_BKP') = 0 AND INSTR(UPPER(TableName),'_CPY') = 0)"\
            "and Lower(DataBaseName) not like '%dmx_ac%'"\
                "Order By database_name, table_name, object_type;".format(schema_prefix)
            objectsmeta_df = pd.read_sql(query, td_engine)
            db_objects = []
            db_objects.append(",".join(["Database","Object","Object_Type"]))

            for index, objectsmeta_rec in objectsmeta_df.iterrows():
                database_name = str(objectsmeta_rec['database_name']).strip().lower()
                object_name = str(objectsmeta_rec['table_name']).strip().lower()
                object_type = str(objectsmeta_rec['object_type']).strip().upper()
                db_objects.append(",".join([database_name, object_name, object_type]))
                if object_type.strip() == 'V':
                    if not os.path.exists(f"{td_output_path_folder}{database_name}"):
                        os.makedirs(f"{td_output_path_folder}{database_name}")
                    view_output_path = f"{td_output_path_folder}{database_name}/{object_name}.sql"
                    view_definition = [str(objectsmeta_rec['view_definition'])]
                    write_file_local(view_output_path, view_definition)
            if not os.path.exists(f"{output_path_folder}"):
                os.makedirs(f"{output_path_folder}")
            output_path = "{}\\{}.{}".format(output_path_folder, schema_prefix, output_file_extension)
            
            write_file_local(output_path, db_objects)

        except Exception as e1:
            print(e1)
            print("Database : {}, Table : {}".format(database_name,object_name))
            pass
        
        dt2 = datetime.now()
        print(dt2-dt1)
        print("Tables for Copying Data : ", len(objectsmeta_df))

In [ ]:
print("Begin of Processing")

gen_db_objects_list_lob()

print("End of Processing")